# Generation of Time series temperature data

- Using TGAN or Time-Series Generative Adversiarial Networks we now try to produce more data
- We implement an unsupervised adversarial loss on both real and synthetic data
- We implement this with tensorflow 2
- While in a Traditional GAN we will have a generator and a discriminator, here we also have recovery and embedder models.

## Network
- The 3 layer GRU nework has been used here, but we can also use an LSTM

### Loading the data

In [8]:
# We load the data
from abc import ABC

import numpy as np
import pandas as pd
df = pd.read_csv('../data/feed1.csv')
df = df.drop(columns='field5')

df_main = pd.read_csv('../data/feed2.csv')
df_main.rename(columns={'field1': 'external', 'field2':'internal'}, inplace=True)
print(df_main.shape)

(100, 4)


In [9]:
from tensorflow.keras.layers import GRU, Dense, LSTM

def make_net(model, n_layers, hidden_units, output_units, net_type='GRU'):
    if net_type=='GRU':
        for i in range(n_layers):
            model.add(GRU(units=hidden_units,
                      return_sequences=True,
                      name=f'GRU_{i + 1}'))
    else:
        for i in range(n_layers):
            model.add(LSTM(units=hidden_units,
                      return_sequences=True,
                      name=f'LSTM_{i + 1}'))

    model.add(Dense(units=output_units,
                    activation='sigmoid',
                    name='OUT'))
    return model

In [11]:
from tensorflow.keras import Sequential, Input
from tensorflow.keras import Model

class Generator(Model):
    def __init__(self, hidden_dim, net_type='GRU'):
        self.hidden_dim = hidden_dim
        self.net_type = net_type

    def build(self, input_shape):
        model = Sequential(name='Generator')
        model.add(Input(shape=input_shape))
        model = make_net(model,
                         n_layers=3,
                         hidden_units=self.hidden_dim,
                         output_units=self.hidden_dim,
                         net_type=self.net_type)
        return model

In [12]:
class Discriminator(Model):
    def __init__(self, hidden_dim, net_type='GRU'):
        self.hidden_dim = hidden_dim
        self.net_type=net_type

    def build(self, input_shape):
        model = Sequential(name='Discriminator')
        model = make_net(model,
                         n_layers=3,
                         hidden_units=self.hidden_dim,
                         output_units=1,
                         net_type=self.net_type)
        return model

In [13]:
class Recovery(Model):
    def __init__(self, hidden_dim, n_seq):
        self.hidden_dim=hidden_dim
        self.n_seq=n_seq
        return

    def build(self, input_shape):
        recovery = Sequential(name='Recovery')
        recovery.add(Input(shape=input_shape, name='EmbeddedData'))
        recovery = make_net(recovery,
                            n_layers=3,
                            hidden_units=self.hidden_dim,
                            output_units=self.n_seq)
        return recovery

In [14]:
class Embedder(Model):

    def __init__(self, hidden_dim):
        self.hidden_dim=hidden_dim
        return

    def build(self, input_shape):
        embedder = Sequential(name='Embedder')
        embedder.add(Input(shape=input_shape, name='Data'))
        embedder = make_net(embedder,
                            n_layers=3,
                            hidden_units=self.hidden_dim,
                            output_units=self.hidden_dim)
        return embedder

In [15]:
class Supervisor(Model):
    def __init__(self, hidden_dim):
        self.hidden_dim=hidden_dim

    def build(self, input_shape):
        model = Sequential(name='Supervisor')
        model.add(Input(shape=input_shape))
        model = make_net(model,
                         n_layers=2,
                         hidden_units=self.hidden_dim,
                         output_units=self.hidden_dim)
        return model

In [22]:
df_test = pd.read_csv('../data/feed2.csv')

In [23]:
randomArray = df_test['field2'].to_numpy()
print(df_test['field2'], randomArray)

0     104.8
1     107.0
2     117.3
3     118.1
4     118.4
      ...  
95    190.4
96    190.7
97    192.0
98    192.9
99    193.3
Name: field2, Length: 100, dtype: float64 [104.8 107.  117.3 118.1 118.4 122.1 141.1 143.7 143.8 145.7 149.9 151.4
 150.6 151.4 150.9 151.8 149.1 149.5 150.2 151.3 151.8 149.  149.3 150.3
 151.3 150.6 151.3 149.4 151.1 154.9 148.9 151.3 152.1 149.3 150.1 149.5
 151.2 149.5 152.1 149.  149.  151.5 151.7 156.7 151.3 150.6 150.8 149.5
 153.5 149.1 149.1 150.  150.9 151.8 151.1 151.1 151.1 151.5 153.9 150.6
 150.  151.9 149.7 150.5 151.3 149.7 151.6 150.6 151.8 150.  152.  152.7
 152.5 153.4 158.  158.8 159.  160.8 161.1 161.2 163.  163.8 165.3 169.2
 172.3 174.8 175.2 176.8 178.7 181.  181.3 181.4 183.7 186.2 188.8 190.4
 190.7 192.  192.9 193.3]


In [26]:
import numpy as np
def MinMaxScaler(data):
    """Min Max normalizer.

    Args:
      - data: original data

    Returns:
      - norm_data: normalized data
    """
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    norm_data = numerator / (denominator + 1e-7)
    return norm_data

In [27]:
norm_data = MinMaxScaler(randomArray)

In [28]:
norm_data


array([0.        , 0.02485876, 0.14124294, 0.15028249, 0.15367232,
       0.19548023, 0.41016949, 0.43954802, 0.44067797, 0.46214689,
       0.50960452, 0.52655367, 0.51751412, 0.52655367, 0.52090395,
       0.53107345, 0.50056497, 0.50508475, 0.51299435, 0.52542373,
       0.53107345, 0.49943503, 0.50282486, 0.51412429, 0.52542373,
       0.51751412, 0.52542373, 0.5039548 , 0.52316384, 0.56610169,
       0.49830508, 0.52542373, 0.53446328, 0.50282486, 0.51186441,
       0.50508475, 0.52429378, 0.50508475, 0.53446328, 0.49943503,
       0.49943503, 0.52768362, 0.5299435 , 0.58644068, 0.52542373,
       0.51751412, 0.51977401, 0.50508475, 0.55028249, 0.50056497,
       0.50056497, 0.51073446, 0.52090395, 0.53107345, 0.52316384,
       0.52316384, 0.52316384, 0.52768362, 0.55480226, 0.51751412,
       0.51073446, 0.53220339, 0.50734463, 0.51638418, 0.52542373,
       0.50734463, 0.52881356, 0.51751412, 0.53107345, 0.51073446,
       0.53333333, 0.54124294, 0.53898305, 0.54915254, 0.60112